# Introduction to graph analysis with networkX

## Graphs are everywhere

<center><img src="img/17_node_mesh_network.png" alt="Routing graph" style="height: 900px;"/></center>

<center><img src="img/Familia_Curie.png" alt="Family tree" style="height: 900px;"/></center>

<center><img src="tmp/social_network.png" alt="Semantic graph" style="height: 900px;"/></center>

<center><img src="img/Semantic_Net.svg" alt="Semantic graph" style="height: 900px;"/></center>

## What are graphs?

### Definition
- A **graph** is a pair G = (V, E), where V is a set whose elements are called **vertices** (singular: vertex, also called nodes), and E is a set of two-sets (set with two distinct elements) of vertices, whose elements are called **edges** (sometimes links or lines)
- A **directed graph** or **digraph** is a graph in which edges have orientations
- A **weighted graph** or a **network** is a graph in which a number (the weight) is assigned to each edge

### Further terms
- **Centrality**: identify the most important vertices within a graph
- **Component**: Is a subgraph in which any two vertices are connected to each other by paths in an undirected graph
- **Complete graph**: Every node is connected to each other node


## Preparation

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
DEFAULT_FIG_SIZE=(18, 14)

In [ ]:
def get_node_size(value_dict):
    value_array = np.array(list(value_dict.values()))
    value_range = value_array.max() - value_array.min()
    node_size = (2 * (value_array - value_array.min())/value_range + 1) * 300
    return node_size.tolist()

In [ ]:
def plot_graph(G, node_size=1000, figsize=DEFAULT_FIG_SIZE, width=2, arrowsize=30, plot_fun=nx.draw_networkx, **kwargs):
    plt.figure(figsize=figsize)
    plot_fun(G, with_labels=True, arrowsize=arrowsize, node_size=node_size, width=width, **kwargs)

## networkX package structure

- **networkx.{Graph, DiGraph, MultiGraph, MultiDiGraph}**: Basic classes for Graphs
- **networkx.algorithms.\***: Functions to evaluate and analysing on a graph structure
- **networkx.classes.function.\***: Get graph properties via function calls
- **networkx.generator.\***: Generate specific types of graphs or random graphs, some existing datasets
- **networkx.linalg.\***: Calculate some derived matrix properties of graph
- **networkx.convert.\***: Conversion from/to different python data types
- **networkx.drawing.\***: (Basic) layouting and plotting functions

## Calling package functions
Almost every function can be used by applying: 
```
nx.function_name(G, additional_arguments)
```
where G is the Graph you are trying to analyse

## Defining graphs in networkX

### Undirected

In [ ]:
G = nx.Graph() 

# Add a node
G.add_node(1) 
G.add_nodes_from([2,3]) # You can also add a list of nodes by passing a list argument

In [ ]:
plot_graph(G, width=3)

In [ ]:
# Add edges 
G.add_edge(1,2)

e = (2,3)
G.add_edge(*e) # * unpacks the tuple
G.add_edges_from([(1,2), (1,3)]) # Just like nodes we can add edges from a list

In [ ]:
plot_graph(G, width=3)

### Directed

In [ ]:
G = nx.DiGraph()

# Defining nodes and edges is the same as in the Graph example:
G.add_nodes_from([1, 2,3])

G.add_edges_from([(1,2), (1,3), (2,3)])

In [ ]:
plot_graph(G, width=3)

### With weights

In [ ]:
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4])
G.add_weighted_edges_from([(1, 4, 5.), (2, 3, 0.5), (1, 2, 1.), (3, 4, 3.)])

In [ ]:
nx.attr_matrix(G, edge_attr='weight')

In [ ]:
labels = nx.get_edge_attributes(G, 'weight')

In [ ]:
pos = nx.spring_layout(G)
plt.figure(figsize=(16, 12)) 
nx.draw_networkx_nodes(G, pos, node_size=600)
nx.draw_networkx_edges(G, pos, width=list(labels.values()))
nx.draw_networkx_labels(G, pos);
nx.draw_networkx_edge_labels(G, pos, font_size=20);

### Accessing graph properties

In [ ]:
G.nodes()

In [ ]:
G.edges()

## Creating a graph with the conversion functions

In [ ]:
edges = pd.read_csv('data/out.moreno_innovation_innovation', sep=' ', names=['from_node', 'to_node'], skiprows=2)
edges.head()

In [ ]:
digraph = nx.from_pandas_edgelist(edges,'from_node', 'to_node', create_using=nx.DiGraph)

In [ ]:
plt.figure(figsize=(22, 14))
plt.imshow((nx.adjacency_matrix(digraph).todense()), aspect=0.6);
plt.tick_params(labelsize=20)

In [ ]:
plot_graph(digraph, node_size=300, width=1, arrowsize=10)

## Random Graphs

### Erdos-Renyi networks
Every edge has constant probabilty $p$

In [ ]:
erdos_renyi = nx.random_graphs.erdos_renyi_graph(50, 0.0784)

In [ ]:
plot_graph(erdos_renyi, node_size=600)

#### Effect of growing n with constant p

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(24, 11))
nx.draw_networkx(nx.random_graphs.erdos_renyi_graph(10, 0.1), with_labels=True, node_size=400, width=2, ax=axes[0])
nx.draw_networkx(nx.random_graphs.erdos_renyi_graph(25, 0.1), with_labels=True, node_size=400, width=2, ax=axes[1])
[[axes[j].set_xticks([]) for j in range(2)]];
[[axes[j].set_yticks([]) for j in range(2)]];

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(24, 11))
nx.draw_networkx(nx.random_graphs.erdos_renyi_graph(50, 0.1), with_labels=True, node_size=400, width=2, ax=axes[0])
nx.draw_networkx(nx.random_graphs.erdos_renyi_graph(100, 0.1), with_labels=True, node_size=400, width=2, ax=axes[1])
[[axes[j].set_xticks([]) for j in range(2)]];
[[axes[j].set_yticks([]) for j in range(2)]];

### $\log(n)/n$ phase transition

In [ ]:
p = np.log(200)/200
p

In [ ]:
# Add plot increasing edge prob multiple trials amount one component vs more than one
probs = np.arange(0.01, 0.05001, 0.0025)
has_one_component = [np.mean([nx.number_connected_components(nx.random_graphs.erdos_renyi_graph(200, p)) < 2 for i in range(200)]) for p in probs]

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(probs, has_one_component, lw=4)
plt.xlabel('Edge probability', fontsize=25)
plt.ylabel('Frequency of one component', fontsize=25)
plt.tick_params(labelsize=20)
plt.axvline(x=p,linewidth=4, color='r')
plt.show()

### Preferential attachment  (Barabási–Albert)
Edge probability is proportional to node degree

In [ ]:
bara_albert = nx.barabasi_albert_graph(50, 2)

In [ ]:
plot_graph(bara_albert, node_size=600, node_color=bara_albert.nodes())

## Graph analysis

In [ ]:
positions = nx.spring_layout(bara_albert)

### Edge density
Number of edges in the graph compared to number of edges in complete graph

In [ ]:
nx.density(bara_albert)

In [ ]:
nx.density(erdos_renyi)

### Dijkstra pathes
Shortest path from one vertex to another vertex

In [ ]:
dij_path = nx.dijkstra_path(bara_albert, source=49, target=20)

In [ ]:
dij_path

In [ ]:
colors = [0]*50
for i in dij_path:
    colors[i]=1

In [ ]:
plot_graph(bara_albert, node_size=600, node_color=colors, pos=positions)

### Average shortest path length
Average length of all shortest pathes

In [ ]:
nx.average_shortest_path_length(bara_albert)

In [ ]:
nx.average_shortest_path_length(erdos_renyi)

### Node degree
Number of nodes a node is connected to

In [ ]:
node_degree = nx.degree_centrality(bara_albert)

In [ ]:
plot_graph(bara_albert, node_size=get_node_size(node_degree), node_color=get_node_size(node_degree), pos=positions)

### Node degree histogramm

In [ ]:
degrees_bara_albert = nx.degree_histogram(nx.barabasi_albert_graph(500, 2))
degrees_erdos_renyi = nx.degree_histogram(nx.random_graphs.erdos_renyi_graph(500, 996./(250.*499.)))

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(list(range(len(degrees_bara_albert))), np.array(degrees_bara_albert)/len(bara_albert.nodes), lw=4)
plt.plot(list(range(len(degrees_erdos_renyi))), np.array(degrees_erdos_renyi)/len(erdos_renyi.nodes), lw=4)
plt.xlabel('Degree', fontsize=25)
plt.ylabel('Degree frequency', fontsize=25)
plt.tick_params(labelsize=20)
plt.legend(['Barabasi-Albert', 'Erdos-Renyi'])
plt.show()

### Closeness centrality
Inverse average distance to all other nodes in the graph

In [ ]:
closeness = nx.closeness_centrality(bara_albert)

In [ ]:
plot_graph(bara_albert, node_size=get_node_size(closeness), node_color=get_node_size(closeness), pos=positions)

### Betweenness centrality
Number of shortest pathes between two nodes the node is contained

In [ ]:
betweenness = nx.betweenness_centrality(bara_albert, normalized=True, endpoints=True)

In [ ]:
plot_graph(bara_albert, node_size=get_node_size(betweenness), node_color=get_node_size(betweenness), pos=positions)

### Eigenvector centrality
- Eigenvector belonging to the largest eigenvalue of the adjacency matrix
- Captures imortance of nodes the node is connected to

In [ ]:
eigenvec_cen = nx.eigenvector_centrality_numpy(bara_albert)

In [ ]:
plot_graph(bara_albert, node_size=get_node_size(eigenvec_cen), node_color=get_node_size(eigenvec_cen), pos=positions)

### Clustering
Fraction of neighboring nodes that have a edge with each other (friends are also friends)

In [ ]:
clustering = nx.clustering(bara_albert)

In [ ]:
plot_graph(bara_albert, node_size=get_node_size(clustering), node_color=get_node_size(clustering), pos=positions)

### Minimum spanning tree
Graph with the smallest amount of edge weights that connects all vertices

In [ ]:
msp = nx.minimum_spanning_tree(bara_albert)

In [ ]:
plot_graph(msp, node_size=600, pos=positions)

## Graph Layouting

### Circular layout
Nodes are just positioned on a circle

In [ ]:
plot_graph(digraph, node_size=600, arrowsize=10, plot_fun=nx.draw_circular, figsize=(16, 12))

In [ ]:
nx.number_connected_components(digraph.to_undirected())

### Kamada Kawai layout
Edges are of more or less equal length and there are as few crossing edges as possible

In [ ]:
plot_graph(digraph, node_size=600, arrowsize=10, plot_fun=nx.draw_kamada_kawai, figsize=(16, 12))

### Spring layout
Position nodes using attracting and repulsive forces

In [ ]:
plot_graph(digraph, node_size=600, arrowsize=10, plot_fun=nx.draw_spring, figsize=(16, 12))

### Spectral layout
Position nodes using the eigenvectors of the graph Laplacian ($ L = D - A$, where A is the adjacency matrix and D is the diagonal matrix)

In [ ]:
plot_graph(digraph, node_size=2000, plot_fun=nx.draw_spectral, figsize=(16, 12))

### Alternative formulation (calculate layout independently)

In [ ]:
pos = nx.kamada_kawai_layout(digraph)
plot_graph(digraph, node_size=600, arrowsize=10, pos=pos)

## Wrap up
- networkX provides unified interface to work with graphs
- Graph data can be read in from multiple formats
- Generators for creating specific graph types or different kind of random graphs
- Large amount of graph analysis functions
- Basic layouting of graphs for plotting included. Can be extended by Graphviz or PyGraphviz
- ...There is much more than we have covered...

<br>
<br>
<br>
<br>
<div><p><center><font size="30">Thank you!</font></center></p></div>
<br>
<br>
<div><p><center><font size="6">The content of the talk can be found here: <a href="https://github.com/rquadrat/network-analysis">https://github.com/rquadrat/network-analysis</a></font></center></p></div>